In [1]:
#!pip install selenium
#!pip install geonamescache
from helper import *
import selenium
import tweepy
import numpy as np
import pandas as pd
import json
from bs4 import BeautifulSoup
from IPython.display import clear_output #clears output
import datetime
import time

API_Key ='KEY'
API_Key_Secret = 'KEY'

Bearer_Token = 'KEY'

Access_Token = 'KEY'
Access_Token_Secret = 'KEY'

In [2]:
# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(API_Key, API_Key_Secret)
# set access to user's access key and access secret
auth.set_access_token(Access_Token, Access_Token_Secret)
# calling the api
api = tweepy.API(auth)
Client = tweepy.Client(bearer_token=Bearer_Token)

### revising the tweets
now that we have let the tweets cook in the oven for a few days, we will revisit them and see how their stats have changed

In [3]:
currentI = 6 # saves us the hussle of changing the number
df = pd.read_csv('data{}.csv'.format(currentI))
del df['Unnamed: 0']
df

,name,age,city,country,gender,acount age,total tweets,followers,following,respectability,...,word count,is quote,hashtags,hashtag count,trending,using trends?,metaData,join_date,date,tweet id
0,streetsforall,NaN,Desert Hot Springs,United States,NaN,13.5,4714,8223,137,0.016661,...,34,True,NaN,0,"['#TabooToken', 'Chargers', 'Herbert', 'Staley...",False,--------,2010-04-06,2023-01-15,1614604722199228416
1,Quersha_,NaN,Charlotte,United States,NaN,14.5,105862,1239,480,0.387409,...,10,False,NaN,0,"['#TabooToken', 'Chargers', 'Good Sunday', 'He...",False,--------,2010-05-18,2023-01-15,1614604717782437893
2,Gatitaconestres,NaN,New Hampshire,United States,NaN,5.0,12600,333,189,0.567568,...,4,True,NaN,0,"['#TabooToken', 'Chargers', 'Herbert', 'Staley...",False,--------,2019-11-12,2023-01-15,1614604717727928320
3,Cheli_Smith,NaN,Upstate New York,United States,NaN,12.8,13567,542,2121,3.913284,...,17,False,"['shortfilmmaking', 'onset', 'cinema', 'tvfilm...",7,"['#TabooToken', 'Chargers', 'Herbert', 'Staley...",False,--------,2011-05-09,2023-01-15,1614604716750536705
4,IgawaPastor,NaN,Mission Viejo,United States,NaN,10.2,5113,150,398,2.653333,...,9,False,NaN,0,"['#TabooToken', 'Chargers', 'Herbert', 'Staley...",False,--------,2014-02-15,2023-01-15,1614604716163342338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9794,arshadzackeriya,NaN,Wellington City,New Zealand,NaN,14.0,1022,289,482,1.667820,...,26,False,"['DevOps', 'DevOpswithZack']",2,"['Paula', 'Chargers', 'Paul Henry', '#UFCVegas...",False,--------,2011-05-24,2023-01-13,1613851183457906688
9795,Ncookie98,NaN,Auckland,New Zealand,NaN,11.3,133568,121,516,4.264463,...,2,False,NaN,0,"['Paula', 'Chargers', 'Paul Henry', 'Perth', '...",False,--------,2012-02-04,2023-01-13,1613850481738280962
9796,erimedi,NaN,Christchurch City,New Zealand,NaN,8.0,10015,190,376,1.978947,...,11,False,NaN,0,"['Paula', 'Chargers', 'Paul Henry', '#UFCVegas...",True,--------,2016-10-12,2023-01-13,1613849326245261312
9797,auralorgasm,NaN,Kapiti Coast District,New Zealand,NaN,9.8,56050,490,412,0.840816,...,4,True,NaN,0,"['Paula', 'Chargers', 'Paul Henry', '#UFCVegas...",False,--------,2015-10-22,2023-01-13,1613849166685560832


we will use the tweets' IDs to find them again

In [4]:
# creates a list of all the ids in string form
tmp = [t for t in df['tweet id']]

for i in range(0,len(tmp)):
    tmp[i] = str(tmp[i])
tmp
len(tmp)

9799

In [5]:
# speperates them into groups of 100's so that it's easier to put them in the search function

all_ids = []
i = 0

while i < len(tmp):
    all_ids.append(tmp[i:i+100])
    i+=100

now that we have a list of all the IDs we will look them up using the API

In [6]:
'''
while i < len(tmp):
    tweets.append(Client.get_tweets(ids = tmp[i:i+100],tweet_fields=["public_metrics"]))
    i+=100

tmp = []
for x in tweets:
    for y in x:
        for z in y:
            tmp.append(z)
'''#accidentally creates a 3D cube so it then sorts it into a 1D arr


#gets the tweets by id
tweetsArr = []

for i in all_ids:
    tweetsArr.append(Client.get_tweets(ids = i,tweet_fields=["public_metrics"]))



C:\Users\User\anaconda3\lib\site-packages\tweepy\client.py:157: RuntimeWarning: Tweet data missing default edit_history_tweet_ids field
  data = [data_type(result) for result in data]


In [7]:
#fixes uncomfortable array placements
tweets = []
for x in tweetsArr:
    tweets.extend(x)

as usual, the twitter API is choosing to be difficult. the get_tweets function returns a very strange array.
it one array thaat contains many sub-arrays of tweets it failed to aquire and tweets that were retrieved succesfully.
they are ordered in a very particular and annoying way; so we will extract all the tweets we were able to get into its own array

In [8]:
errors = tweets[2::4]#get a list of all the tweets which returned error
tweets = tweets[0::4]#gets the actual tweets

In [9]:
#fixes bad placements again
tmp = []
for i in tweets:
    tmp.extend(i)

tweets = tmp 

In [10]:
name = []
age = []
city = []
country = []
gender = []
acount_age = []
total_tweets = []
followers = []
following = []
respectability = []
verified = []
tweet = []
likes = []
comments = []
retweets = []
Qretweets = []
word_count = []
is_quote = []
hashtags = []
hashtag_count = []
trending = []
using_trends = []
links = []
views = []

now we will create a new DataFrame with updated stats

In [11]:
for t in tweets:
    ID = t['id']
    row = df[df['tweet id'] == ID]
    
    name.append(row['name'].values[0])
    age.append(row['age'].values[0])
    city.append(row['city'].values[0])
    country.append(row['country'].values[0])
    gender.append(row['gender'].values[0])
    acount_age.append(row['acount age'].values[0])
    total_tweets.append(row['total tweets'].values[0])
    followers.append(row['followers'].values[0])
    following.append(row['following'].values[0])
    respectability.append(row['respectability'].values[0])
    verified.append(row['verified'].values[0])
    tweet.append(row['tweet'].values[0])
    word_count.append(row['word count'].values[0])
    is_quote.append(row['is quote'].values[0])
    hashtags.append(row['hashtags'].values[0])
    hashtag_count.append(row['hashtag count'].values[0])
    trending.append(row['trending'].values[0])
    using_trends.append(row['using trends?'].values[0])
    links.append('https://www.twitter.com/'+row['name'].values[0]+'/status/'+str(ID))
    
    likes.append(t['public_metrics']['like_count'])
    views.append(t['public_metrics']['impression_count'])
    retweets.append(t['public_metrics']['retweet_count'] + t['public_metrics']['quote_count'])
    Qretweets.append(t['public_metrics']['quote_count'])
    comments.append(t['public_metrics']['reply_count'])

In [12]:
max_len = len(using_trends)

In [13]:
dic = {'name':name[:max_len],
       'age':age[:max_len],
       'city':city[:max_len],
       'country':country[:max_len],
       'gender':gender[:max_len],
       'acount age':acount_age[:max_len],
       'total tweets':total_tweets[:max_len],
       'followers':followers[:max_len],
       'following':following[:max_len],
       'respectability':respectability[:max_len],
       'verified':verified[:max_len],
       'text':tweet[:max_len],
       'views':views[:max_len],
       'likes':likes[:max_len],
       'retweets':retweets[:max_len],
       'quote retweets':Qretweets[:max_len],
       'comments':comments[:max_len],
       'word count':word_count[:max_len],
       'is quote':is_quote[:max_len],
       'hashtags':hashtags[:max_len],
       'hashtag count':hashtag_count[:max_len],
       'trending':trending[:max_len],
       'using trends?':using_trends[:max_len],
       'link to tweet':links[:max_len]
      }

new_df = pd.DataFrame(dic)

In [14]:
new_df

,name,age,city,country,gender,acount age,total tweets,followers,following,respectability,...,retweets,quote retweets,comments,word count,is quote,hashtags,hashtag count,trending,using trends?,link to tweet
0,streetsforall,NaN,Desert Hot Springs,United States,NaN,13.5,4714,8223,137,0.016661,...,11,0,3,34,True,NaN,0,"['#TabooToken', 'Chargers', 'Herbert', 'Staley...",False,https://www.twitter.com/streetsforall/status/1...
1,Gatitaconestres,NaN,New Hampshire,United States,NaN,5.0,12600,333,189,0.567568,...,0,0,0,4,True,NaN,0,"['#TabooToken', 'Chargers', 'Herbert', 'Staley...",False,https://www.twitter.com/Gatitaconestres/status...
2,Cheli_Smith,NaN,Upstate New York,United States,NaN,12.8,13567,542,2121,3.913284,...,0,0,0,17,False,"['shortfilmmaking', 'onset', 'cinema', 'tvfilm...",7,"['#TabooToken', 'Chargers', 'Herbert', 'Staley...",False,https://www.twitter.com/Cheli_Smith/status/161...
3,IgawaPastor,NaN,Mission Viejo,United States,NaN,10.2,5113,150,398,2.653333,...,0,0,0,9,False,NaN,0,"['#TabooToken', 'Chargers', 'Herbert', 'Staley...",False,https://www.twitter.com/IgawaPastor/status/161...
4,WalshJesuit,NaN,Walsh Jesuit High School,United States,NaN,16.1,1960,3934,484,0.123030,...,7,2,1,41,False,['PartnersinEducation'],1,"['#TabooToken', 'Chargers', 'Herbert', 'Staley...",False,https://www.twitter.com/WalshJesuit/status/161...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9301,arshadzackeriya,NaN,Wellington City,New Zealand,NaN,14.0,1022,289,482,1.667820,...,0,0,0,26,False,"['DevOps', 'DevOpswithZack']",2,"['Paula', 'Chargers', 'Paul Henry', '#UFCVegas...",False,https://www.twitter.com/arshadzackeriya/status...
9302,Ncookie98,NaN,Auckland,New Zealand,NaN,11.3,133568,121,516,4.264463,...,0,0,0,2,False,NaN,0,"['Paula', 'Chargers', 'Paul Henry', 'Perth', '...",False,https://www.twitter.com/Ncookie98/status/16138...
9303,erimedi,NaN,Christchurch City,New Zealand,NaN,8.0,10015,190,376,1.978947,...,0,0,0,11,False,NaN,0,"['Paula', 'Chargers', 'Paul Henry', '#UFCVegas...",True,https://www.twitter.com/erimedi/status/1613849...
9304,auralorgasm,NaN,Kapiti Coast District,New Zealand,NaN,9.8,56050,490,412,0.840816,...,0,0,0,4,True,NaN,0,"['Paula', 'Chargers', 'Paul Henry', '#UFCVegas...",False,https://www.twitter.com/auralorgasm/status/161...


and we will save it

In [15]:
new_df.to_csv('out{}.csv'.format(currentI))